In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sqlfunctions import get_datapunten_voor_model


## Notities
Datapunten AI:
- omzet
- Beursgenoteerd of niet
- sector (nacebel code)
- aantal personeelsleden
- hoofdsteden provincies

In [5]:
# Zet alle data in pd dataframe
dataframe = pd.DataFrame(columns=['ondernemingnr', 'omzet' ,'beursgenoteerd', 'sector', 'personeelsleden', 'hoofdsteden_provincies'])
dataframe.set_index('ondernemingnr', inplace=True)
# TODO: Hoe kom ik aan hoofdsteden_provincies?
data = get_datapunten_voor_model()

,omzet,beursgenoteerd,sector,personeelsleden,hoofdsteden_provincies
ondernemingnr,,,,,


In [7]:
data
# dataframe
# add data to dataframe
for ondernemingnr, omzet, beursgenoteerd, sector, personeelsleden in data:
    dataframe.loc[ondernemingnr] = [omzet, beursgenoteerd, sector, personeelsleden, None]
    # todo: Dit duurt anderhalve minuut, mss nog wat sneller maken ookal moet dat maar 1 keer ferund worden

In [8]:
dataframe

# most efficient way to convert a list of lists to a dataframe
# https://stackoverflow.com/questions/18691084/what-is-the-most-efficient-way-to-convert-a-list-of-lists-to-a-dataframe-in-pytho

,omzet,beursgenoteerd,sector,personeelsleden,hoofdsteden_provincies
ondernemingnr,,,,,
0404935507,0.0,False,Groothandel in diamant en andere edelstenen,15.0,None
0412527142,0.0,False,Overige vervoerondersteunende activiteiten,10.0,None
0461054460,0.0,False,"Groothandel in vaste, vloeibare en gasvormige ...",22.0,None
0632455341,0.0,False,"Groothandel in vaste, vloeibare en gasvormige ...",9.0,None
0475849633,0.0,False,Overige adviesbureaus op het gebied van bedrij...,10.0,None
...,...,...,...,...,...
0899773285,0.0,False,Chapewerken,9.0,None
0899789222,0.0,False,Ingenieurs en aanverwante technische adviseurs...,27.0,None
0899886915,0.0,False,Dakwerkzaamheden,10.0,None
